In [ ]:
import os
from datetime import datetime

import torch
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter
import numpy as np
from tqdm import tqdm
from PIL import Image

from denoisingheat.utils.utils import (
    gen_goals, overlay_goal, randgen_obstacle_masks, randgen_fix_obstacle_masks, draw_obstacles_pixel, load_config, overlay_goals_with_prompts, convert_to_obstacle_masks, draw_obstacles_pil
)
import matplotlib.pyplot as plt
from typing import Optional
import shutil

# Args
config_dir = "./heat_diffusion.yaml"
args = load_config(config_dir)
device = args['device']

bg = Image.open('assets/toy_exp/image.png') # need to change 


img_size = 96
goal_bounds = args['goal_bounds']
agent_bounds = args['agent_bounds']
obstacle_pos = args['drop_region']

model_path = os.path.join(args['log_path'], args['model_path'])

u0 = args['u0']
min_heat_step = args['min_heat_step']
max_heat_step = args['max_heat_step']
noise_steps = 20
sample_num = 3
time_type = args['time_type']

iterations = args['iterations']
train_lr = args['train_lr']
batch_size = 5

goal_num = 3

from transformers import CLIPTokenizer, CLIPTextModel
tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-base-patch32")
text_encoder = CLIPTextModel.from_pretrained("openai/clip-vit-base-patch32")

def get_text_embeds(texts: list):
    # texts: ["apple", "banana", ...]
    # return: (B, seq_len, hidden_dim)
    tokens = tokenizer(texts, padding="max_length", max_length=10, return_tensors="pt")
    with torch.no_grad():
        outputs = text_encoder(**tokens)

    text_embeds = outputs.last_hidden_state
    return text_embeds

# need to change 
goal_image_paths = [
    'assets/toy_exp/apple.png',
    'assets/toy_exp/banana.png',
    'assets/toy_exp/clock.png',
    'assets/toy_exp/blue_cup.png',
    'assets/toy_exp/trash_can.png',
    'assets/toy_exp/garbage1.png',
    'assets/toy_exp/apple.png'
]

goal_images = [Image.open(path) for path in goal_image_paths]
original_images = goal_images.copy()
base_size = original_images[0].size

goal_images_resized = [img.resize(base_size, Image.LANCZOS) for img in goal_images]
# need to change 
goal_prompts = [
    "Move to the apple.",
    "Move to the banana.",
    "Move to the clock.",
    "Move to the blue cup.",
    "Move to the trash can.",
    "Move to the garbage.",
    "Move to the apple."
]


/home/jebeom/miniconda3/envs/lgdh/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/jebeom/LGDH/denoisingheat/models/ddpm/denoising_diffusion.py:846: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @autocast(enabled = False)


In [ ]:
sel_idx = torch.multinomial(          # (B, k)
    torch.ones(batch_size, goal_num, device=device),  
    num_samples=1,
    replacement=False
)
flat_sel   = sel_idx.reshape(-1)                      # (B*k,)
batch_ids  = torch.arange(batch_size, device=device) \
                .unsqueeze(1).expand(-1, 1).reshape(-1)   # (B*k,)

# need to change to real obstacle mask
obstacle_masks = randgen_obstacle_masks(batch_size*1, img_size, device=device)
background = draw_obstacles_pixel(bg, obstacle_masks)
multi_goals = gen_goals(goal_bounds, (goal_num, batch_size), img_size, obstacles=obstacle_masks[:], device=device)
multi_goal = (torch.rand(batch_size, goal_num, 2, device=device, dtype=torch.float32) * 0.2 - 0.1) * 0.05 + multi_goals
# need to change ?
obs1, prompts1 = overlay_goals_with_prompts(background, img_size, goal_images_resized, multi_goal, goal_prompts, goal_size=16)
obs  = obs1[batch_ids]

print(np.shape(obs))
plt.imshow(obs.permute(0,2,3,1)[0].cpu().numpy())
plt.show()
plt.imshow(obs.permute(0,2,3,1)[1].cpu().numpy())
plt.show()
plt.imshow(obs.permute(0,2,3,1)[2].cpu().numpy())
plt.show()
plt.imshow(obs.permute(0,2,3,1)[3].cpu().numpy())
plt.show()
plt.imshow(obs.permute(0,2,3,1)[4].cpu().numpy())
plt.show()
plt.imshow(obs.permute(0,2,3,1)[5].cpu().numpy())
plt.show()
plt.imshow(obs.permute(0,2,3,1)[6].cpu().numpy())
plt.show()

NameError: name 'torch' is not defined